# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846822103458                   -0.70    4.5         
  2   -7.852302092044       -2.26       -1.53    1.0   31.0ms
  3   -7.852614443193       -3.51       -2.55    1.5   33.5ms
  4   -7.852645953057       -4.50       -2.87    2.8   42.5ms
  5   -7.852646496777       -6.26       -3.17    1.0   31.3ms
  6   -7.852646679824       -6.74       -4.08    1.0   31.0ms
  7   -7.852646686662       -8.17       -5.10    2.0   40.7ms
  8   -7.852646686723      -10.21       -5.42    1.8   37.1ms
  9   -7.852646686729      -11.21       -6.00    1.2   33.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846724557801                   -0.70           4.5         
  2   -7.852553399518       -2.23       -1.63   0.80    2.0    339ms
  3   -7.852639399202       -4.07       -2.74   0.80    1.0   28.1ms
  4   -7.852646575014       -5.14       -3.37   0.80    1.8   35.3ms
  5   -7.852646679106       -6.98       -4.25   0.80    1.2    125ms
  6   -7.852646686604       -8.13       -4.81   0.80    2.2   36.2ms
  7   -7.852646686723       -9.93       -5.72   0.80    1.2   30.4ms
  8   -7.852646686730      -11.16       -7.16   0.80    2.2   35.3ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.374758e+01     3.973042e+00
 * time: 0.4497530460357666
     1     1.390111e+00     2.159902e+00
 * time: 0.673893928527832
     2    -1.281260e+00     2.581731e+00
 * time: 0.703524112701416
     3    -3.549023e+00     2.263422e+00
 * time: 0.7468369007110596
     4    -4.693272e+00     2.043200e+00
 * time: 0.789370059967041
     5    -6.622422e+00     1.027231e+00
 * time: 0.8327810764312744
     6    -7.384739e+00     4.601704e-01
 * time: 0.8757550716400146
     7    -7.680842e+00     1.672786e-01
 * time: 0.905689001083374
     8    -7.762391e+00     1.202458e-01
 * time: 0.9353930950164795
     9    -7.794198e+00     9.425547e-02
 * time: 0.9657280445098877
    10    -7.817966e+00     1.052326e-01
 * time: 0.9962821006774902
    11    -7.836844e+00     7.002851e-02
 * time: 1.0267109870910645
    12    -7.846191e+00     3.502849e-02
 * time: 1.0574800968170166
    13    -7.850858e+00     2.438761e-02
 * time: 1.08853197097778

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846897415551                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645908260                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.81s
  3   -7.852646686730      -13.31       -7.24    186ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.154400760943968e-7
|ρ_newton - ρ_scfv| = 7.272914224421655e-8
|ρ_newton - ρ_dm|   = 1.0484725732063504e-9
